In [3]:
import numpy 
import pandas as pd

In [4]:
supplemental_stock_prices = pd.read_csv('ds/supplemental_files/stock_prices.csv')
train_stock_prices = pd.read_csv('ds/train_files/stock_prices.csv')
test_stock_prices = pd.read_csv('ds/example_test_files/stock_prices.csv')

In [5]:
def calc_rate_of_change(current_value,old_value):
    return ((current_value - old_value)/old_value)

In [6]:
def create_base_long_short(prices):
    if(prices == "train"):
        model =  train_stock_prices.copy()
        model["daily_return"] = (model["Close"]-model["Open"])/model["Open"]
        #Open high low close not available
        model=model[model["Date"]!="2020-10-01"]
        model["daily_return"] = model.groupby(["Date"])["daily_return"].apply(lambda x: x.fillna(x.mean()))
        model["Previous Daily Return"] =model.groupby(["SecuritiesCode"])["daily_return"].shift(1)
        model["Previous Daily Return"] =model.groupby(["SecuritiesCode"])["Previous Daily Return"].apply(lambda x: x.fillna(x.mean()))
        model["Rank"] = model.groupby(["Date"])["Previous Daily Return"].rank(method="first")-1
        #Remove first day since no previous daily returns
        model = model[model["Date"]!="2017-01-04"]
        model = model.sort_values(["Date","Rank"]).reset_index(drop=True)
        return model
    elif(prices=="supplemental"):
        model =  pd.concat([train_stock_prices[train_stock_prices["Date"]=="2021-12-03"].copy(),supplemental_stock_prices.copy()])
        model["daily_return"] = (model["Close"]-model["Open"])/model["Open"]
        #Open high low close not available
        model["daily_return"] = model.groupby(["Date"])["daily_return"].apply(lambda x: x.fillna(x.mean()))
        model["Previous Daily Return"] =model.groupby(["SecuritiesCode"])["daily_return"].shift(1)
        model["Rank"] = model.groupby(["Date"])["Previous Daily Return"].rank(method="first")-1
        #Remove first day since no previous daily returns
        model = model[model["Date"]!="2021-12-03"]
        model = model.sort_values(["Date","Rank"]).reset_index(drop=True)
        return model
    else:
        model = pd.concat([train_stock_prices[train_stock_prices["Date"]=="2021-12-03"].copy(),test_stock_prices.copy()])
        model["daily_return"] = (model["Close"]-model["Open"])/model["Open"]
        model["Closing Price after 1 day"] = model["Close"].shift(-1)
        model["Closing Price after 2 days"] = model["Close"].shift(-2)
        model["Target"] = model.apply( lambda x: calc_rate_of_change(x["Closing Price after 2 days"],x["Closing Price after 1 day"]),axis=1)
        model["daily_return"] = (model["Close"]-model["Open"])/model["Open"]
        #Open high low close not available
        model["daily_return"] = model.groupby(["Date"])["daily_return"].apply(lambda x: x.fillna(x.mean()))
        model["Previous Daily Return"] =model.groupby(["SecuritiesCode"])["daily_return"].shift(1)
        model["Rank"] = model.groupby(["Date"])["Previous Daily Return"].rank(method="first")-1
        #Remove first day since no previous daily returns
        model = model[model["Date"]!="2021-12-03"]
        model = model.sort_values(["Date","Rank"]).reset_index(drop=True)
        return model

In [7]:
standard = create_base_long_short("supplemental")
standard = standard.set_index(["Date","SecuritiesCode"])
standard["Rank"] = standard["Rank"].astype("int")


In [10]:
standard

RowId    Open    High     Low   Close  \
Date       SecuritiesCode                                                  
2021-12-06 2158            20211206_2158  2315.0  2589.0  2112.0  2560.0   
           4080            20211206_4080  1550.0  1635.0  1443.0  1522.0   
           6378            20211206_6378   949.0   962.0   914.0   930.0   
           3563            20211206_3563  4335.0  4410.0  4190.0  4335.0   
           4369            20211206_4369  3560.0  3590.0  3440.0  3505.0   
...                                  ...     ...     ...     ...     ...   
2022-04-28 9533            20220428_9533  2710.0  3055.0  2693.0  3055.0   
           3154            20220428_3154   997.0  1004.0   976.0  1004.0   
           9919            20220428_9919  1375.0  1389.0  1331.0  1331.0   
           6516            20220428_6516  5270.0  5670.0  5250.0  5610.0   
           4880            20220428_4880  4400.0  4525.0  3860.0  3860.0   

                             Volume  AdjustmentFactor  ExpectedDividend  \
Date       SecuritiesCode                                                 
2021-12-06 2158            12769400               1.0               NaN   
           4080             2002800               1.0               NaN   
           6378              759700               1.0               NaN   
           3563             1354400               1.0               NaN   
           4369              180700               1.0               NaN   
...                             ...               ...               ...   
2022-04-28 9533              903200               1.0               NaN   
           3154               38700               1.0               NaN   
           9919               86100               1.0               NaN   
           6516              407500               1.0               NaN   
           4880             1182300               1.0               NaN   

                           SupervisionFlag    Target  daily_return  \
Date       SecuritiesCode                                            
2021-12-06 2158                      False  0.036559      0.105832   
           4080                      False -0.006506     -0.018065   
           6378                      False  0.027689     -0.020021   
           3563                      False  0.001112      0.000000   
           4369                      False  0.021563     -0.015449   
...                                    ...       ...           ...   
2022-04-28 9533                      False  0.030822      0.127306   
           3154                      False  0.015075      0.007021   
           9919                      False  0.037037     -0.032000   
           6516                      False  0.030741      0.064516   
           4880                      False -0.010540     -0.122727   

                           Previous Daily Return  Rank  
Date       SecuritiesCode                               
2021-12-06 2158                        -0.238325     0  
           4080                        -0.179728     1  
           6378                        -0.079612     2  
           3563                        -0.059259     3  
           4369                        -0.052069     4  
...                                          ...   ...  
2022-04-28 9533                         0.102729  1993  
           3154                         0.107875  1994  
           9919                         0.155116  1995  
           6516                         0.160932  1996  
           4880                         0.176774  1997  

[195993 rows x 13 columns]

In [6]:
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    sample_prediction["Rank"]=sample_prediction.apply(lambda x: standard.loc[(x["Date"],x["SecuritiesCode"])]["Rank"] , axis=1)
    env.predict(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
